# World Bank International Debt Analysis

**Dataset:** The dataset belongs to the __[World Bank](https://www.worldbank.org/en/home)__ and contains information about how much money countries worldwide owing to the Bank.

Just like citizens go to a bank if they need money to grow economically, countries also approach the World Bank for loans to support development in their countries. In this project, we will explore countries' debts to the World Bank using SQL. We will find the answers to questions like:

1. What is the total amount of money that all countries owe to the World Bank?
2. Which country has the highest debt, how much is that, and the percentage of the worlds debt?
3. What is the mean debt owed by countries for different debt indicators?
4. What is the maximum debt held by the top countries by indicator type?

## Connecting to Google BigQuery
Code to make the connection to the `world_bank_intl_debt` database where the table `internation_debt` is located. 

In [9]:
#Set environment variables for your notebook
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/jasonriggins/Downloads/eco-plating-232402-b2252bb35aa4.json'
#Imports google cloud client library and initiates BQ service
from google.cloud import bigquery
bigquery_client = bigquery.Client()

The first thing we need to do is select everything in the table to understand how the data is layed out. Let's limit that output to take a quick look.

In [10]:
#Write Query on BQ
QUERY = """

SELECT *
FROM
  `bigquery-public-data.world_bank_intl_debt.international_debt`
  ORDER BY country_name
  LIMIT 10

  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df

,country_name,country_code,indicator_name,indicator_code,value,year
0,Afghanistan,AFG,IDA grants (current US$),DT.DIS.IDAG.CD,NaN,<NA>
1,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,NaN,<NA>
2,Afghanistan,AFG,IDA grants (current US$),DT.DIS.IDAG.CD,NaN,<NA>
3,Afghanistan,AFG,IDA grants (current US$),DT.DIS.IDAG.CD,NaN,<NA>
4,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,NaN,<NA>
5,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,NaN,<NA>
6,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,NaN,<NA>
7,Afghanistan,AFG,IDA grants (current US$),DT.DIS.IDAG.CD,NaN,<NA>
8,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,NaN,<NA>
9,Afghanistan,AFG,IDA grants (current US$),DT.DIS.IDAG.CD,NaN,<NA>


## 1. What is the total amount of money owed to the world bank?
After looking at the results we can see that Afghanistan would have debt amounts with a different `indicator_name`. But, we don't know how many different countries or indicator names we are working with. Let's find out how many countries there are, the total of the worlds debt.

In [11]:
QUERY = """

SELECT 
    COUNT(DISTINCT country_name) AS total_distinct_countries,
    CAST(SUM(value) AS int) AS total_debt,
FROM
  `bigquery-public-data.world_bank_intl_debt.international_debt`

  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df

,total_distinct_countries,total_debt
0,135,316944895604019


As we can see form the query above there are 135 countries and a total debt of 316,944,895,604,020 USD on the table. That's a total world debt of 317 trillion dollars.

## Finding the distinct debt indicators

Lets take a look at the debt indicators and codes to see how many there are.

In [12]:
QUERY = """

SELECT 
    COUNT(DISTINCT indicator_name) AS number_of_indicator_names, 
    COUNT(DISTINCT indicator_code) AS number_of_indicator_codes
FROM
  `bigquery-public-data.world_bank_intl_debt.international_debt`

  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df

,number_of_indicator_names,number_of_indicator_codes
0,497,497


It seems safe to assume that the indicator codes are directly related to the indicator names. We will be using the `indicator_names` columnin the rest of the analysis to make things easier to read.

## 2. Which country has the highest debt, how much is that, and the percentage of the worlds debt?


Now that we have an idea of what numbers we are working with let's see which countries are carrying the most debt and the average amounts of the top debt indicator.

<div class="alert alert-block alert-info">
The - country_name NOT LIKE '%income' - line of code was used to eleminate some generic country names that included multiple small countries. This elimated data associated with names like: Low & middle income, Middle income, Upper middle income, and Lower middle income
</div>


In [13]:
QUERY = """

SELECT
    country_name,
    CAST(SUM(value) AS int) AS total_debt,
    ROUND(CAST(SUM(value) AS int) / 316944895604019 * 100,2) AS percent_of_world_debt
FROM
    `eco-plating-232402.world_bank_intl_debt.international_ddebt`
    WHERE
        country_name NOT LIKE '%income'
    GROUP BY country_name
    ORDER BY total_debt DESC
LIMIT 10

  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df

,country_name,total_debt,percent_of_world_debt
0,Latin America & Caribbean (excluding high income),21586527231259,6.81
1,Europe & Central Asia (excluding high income),12470638299106,3.93
2,East Asia & Pacific (excluding high income),9526097308235,3.01
3,South Asia,7518452126433,2.37
4,Sub-Saharan Africa (excluding high income),7089309388594,2.24
5,Mexico,6261206147616,1.98
6,IDA only,5934219325574,1.87
7,Least developed countries: UN classification,5489405066449,1.73
8,India,4476825177261,1.41
9,Brazil,3859450893962,1.22


The country Latin America & Caribbean is carrying the most debt of any country at 21,586,527,231,259 USD. That is 6.81 percent of the worlds debt and almost double the amout of debt of the next highest country.

## 3. What is the mean debt owed by countries for different debt indicators?


In [14]:
QUERY = """

SELECT
    indicator_name AS debt_type,
    CAST(AVG(value) AS int) AS indicator_average,   
    ROUND(CAST(SUM(value) AS int) / 316944895604019 * 100,2) AS percent_of_world_debt
FROM
    `eco-plating-232402.world_bank_intl_debt.international_ddebt`
    WHERE
        country_name NOT LIKE '%income'
    GROUP BY debt_type
    ORDER BY indicator_average DESC
LIMIT 10

  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df

,debt_type,indicator_average,percent_of_world_debt
0,"Principal repayments on external debt, long-te...",4409173910,10.13
1,"Principal repayments on external debt, public ...",2841935625,6.53
2,"Principal repayments on external debt, private...",1567238285,3.60
3,"PPG, official creditors (AMT, current US$)",1474301478,3.39
4,"PPG, private creditors (AMT, current US$)",1367634147,3.14
5,"Interest payments on external debt, long-term ...",1308319556,3.01
6,"PPG, bonds (AMT, current US$)",1131161658,2.60
7,"PPG, multilateral (AMT, current US$)",852352833,1.96
8,"PPG, private creditors (INT, current US$)",648632200,1.49
9,"PPG, bilateral (AMT, current US$)",621948646,1.43



The highest indicator is Principal repayments on external debt, long-term at an overall average of 4,409,173,910 USD. This indicator accounts for 10.13 percent of the world's total debt.



## 4. What is the maximum by indicator type in respect to country names?

In [15]:
QUERY = """
    SELECT
        country_name,
        indicator_name,
        CAST(Max(value) AS int) AS maximum_debt
    FROM
        `eco-plating-232402.world_bank_intl_debt.international_ddebt`
    WHERE
        country_name NOT LIKE '%income'
    GROUP BY country_name, indicator_name
    ORDER BY maximum_debt DESC
    LIMIT 10

  """
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
df = Query_Results.to_dataframe()
#View top few rows of result
df

,country_name,indicator_name,maximum_debt
0,Latin America & Caribbean (excluding high income),"Principal repayments on external debt, long-te...",82521070084
1,Europe & Central Asia (excluding high income),"Principal repayments on external debt, long-te...",63558133854
2,Latin America & Caribbean (excluding high income),"Principal repayments on external debt, public ...",59445236262
3,Latin America & Caribbean (excluding high income),"PPG, private creditors (AMT, current US$)",43513895536
4,East Asia & Pacific (excluding high income),"Principal repayments on external debt, long-te...",43480460753
5,Latin America & Caribbean (excluding high income),"PPG, bonds (AMT, current US$)",40118171134
6,Latin America & Caribbean (excluding high income),"Interest payments on external debt, long-term ...",36214576148
7,South Asia,"Principal repayments on external debt, long-te...",35177855642
8,Europe & Central Asia (excluding high income),"Principal repayments on external debt, private...",31905991216
9,Europe & Central Asia (excluding high income),"Principal repayments on external debt, public ...",31652142638


According to the query Latin America & Caribbean and Europe & Central Asia with the debt type of Principal repayments on external debt, long-term are within 21 billion USD of each other.